In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect('db')

In [3]:
data = [[3, '2020-03-21 10:16:13'], [7, '2020-01-04 13:57:59'], [2, '2020-07-29 23:09:44'], [6, '2020-12-09 10:39:37']]
signups = pd.DataFrame(data, columns=['user_id', 'time_stamp']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]'})
data = [[3, '2021-01-06 03:30:46', 'timeout'], [3, '2021-07-14 14:00:00', 'timeout'], [7, '2021-06-12 11:57:29', 'confirmed'], [7, '2021-06-13 12:58:28', 'confirmed'], [7, '2021-06-14 13:59:27', 'confirmed'], [2, '2021-01-22 00:00:00', 'confirmed'], [2, '2021-02-28 23:59:59', 'timeout']]
confirmations = pd.DataFrame(data, columns=['user_id', 'time_stamp', 'action']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]', 'action':'object'})

In [4]:
signups.to_sql('signups', con, index=False, if_exists='replace')
confirmations.to_sql('confirmations', con, index=False, if_exists='replace')

In [67]:
sql = ''' 
        select user_id, round(avg(s), 2) as confirmation_rate
        from
            (select 
                user_id,
                action, 
                case 
                    when action = "confirmed" then 1 
                    else 0
                end as s
            from signups
            left join confirmations using(user_id)) t1
        group by user_id
            
'''

In [68]:
pd.read_sql(sql, con)

,user_id,confirmation_rate
0,2,0.5
1,3,0.0
2,6,0.0
3,7,1.0
